In [18]:
# Save the trained model and the scaler
print("Saving model and scaler...")
joblib.dump(xgb_model, 'xgboost_model.joblib')
joblib.dump(scaler, 'scaler.joblib')
print("Files saved successfully.")

Saving model and scaler...
Files saved successfully.


1. Setup and Load the Trained Model

First, we import the necessary libraries, including ipywidgets for the interactive controls. Then, we load the XGBoost model and the standard scaler that we saved at the end of Notebook 3.

In [19]:
import joblib
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Load the trained model and the scaler
print("Loading model and scaler...")
model = joblib.load('xgboost_model.joblib')
scaler = joblib.load('scaler.joblib')
print("Files loaded successfully.")

Loading model and scaler...
Files loaded successfully.


2. Build the Interactive Dashboard

Now, we'll create the user interface. We will use sliders for each input parameter. These sliders will be linked to a function that takes their values, scales them, feeds them to the model, and displays the predicted $k_{eff}$ in real-time.

In [20]:
# 1. Define the sliders for each reactor parameter
# The min, max, and step values are based on the original simulation ranges.
style = {'description_width': 'initial'}
enrichment_slider = widgets.FloatSlider(
    value=3.0, min=1.0, max=5.0, step=0.1,
    description='Uranium Enrichment (%):', style=style, continuous_update=False
)
radius_slider = widgets.FloatSlider(
    value=0.42, min=0.35, max=0.50, step=0.01,
    description='Fuel Pin Radius (cm):', style=style, continuous_update=False
)
cladding_slider = widgets.FloatSlider(
    value=0.065, min=0.05, max=0.08, step=0.001,
    description='Cladding Thickness (cm):', style=style, continuous_update=False,
    readout_format='.3f'
)
pitch_slider = widgets.FloatSlider(
    value=1.35, min=1.1, max=1.6, step=0.01,
    description='Lattice Pitch (cm):', style=style, continuous_update=False
)

# 2. Define an output widget to display the results
output_keff = widgets.HTML(value="<font size='+2'><b>Predicted k_eff: </b></font>")
output_status = widgets.HTML(value="<font size='+1'><b>Status: </b>")

# 3. Define the prediction function that links sliders to the model
def predict_on_change(enr, rad, clad, pitch):
    # Assemble the input features into a 2D array (model expects it)
    input_features = np.array([[enr, rad, clad, pitch]])

    # Scale the features using the loaded scaler
    input_scaled = scaler.transform(input_features)

    # Make a prediction with the loaded model
    prediction = model.predict(input_scaled)
    keff = prediction[0]

    # Determine the reactor's status
    if keff > 1.005:
        status = "<font color='red'>Supercritical</font>"
    elif keff < 0.995:
        status = "<font color='blue'>Subcritical</font>"
    else:
        status = "<font color='green'>Critical</font>"

    # Update the output widgets with the new values
    output_keff.value = f"<font size='+2'><b>Predicted k_eff: {keff:.5f}</b></font>"
    output_status.value = f"<font size='+1'><b>Status: </b>{status}</font>"

# 4. Use widgets.interactive to link the function and the sliders
# This creates a "view" that automatically calls the function when a slider changes.
interactive_view = widgets.interactive_output(
    predict_on_change,
    {'enr': enrichment_slider, 'rad': radius_slider, 'clad': cladding_slider, 'pitch': pitch_slider}
)

# 5. Arrange the widgets in a user-friendly layout
# VBox for vertical layout, HBox for horizontal
controls = widgets.VBox([enrichment_slider, radius_slider, cladding_slider, pitch_slider])
outputs = widgets.VBox([output_keff, output_status])
dashboard = widgets.VBox([
    widgets.HTML("<h3>Reactor Criticality Surrogate Model</h3>"),
    controls,
    outputs,
    interactive_view # This is hidden but links everything
])

# Call the function once to initialize the output
predict_on_change(enrichment_slider.value, radius_slider.value, cladding_slider.value, pitch_slider.value)

print("Dashboard ready. Move the sliders to see instant predictions.")

# Display the dashboard
display(dashboard)

Dashboard ready. Move the sliders to see instant predictions.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
